# Behind the Eyes: Insights into Cognition from Naturalistic Gaze Behavior in VR
## Introduction
In this tutorial, we will be learning how to analyze eye-tracking data from a virtual reality scene viewing experiment using vrGazeCore, a toolbox built for processing eye-tracking data collected in 360 degree virtual environments.

In this tutorial we will:
1. Load and set parameters for conducting an eye-tracking analysis
2. Analyze  gaze behavior for a single subject and scene
3. Look at how gaze behavior progresses over the time course for a single subject and scene
4. Analyze gaze behavior from a group of subjects
5. Apply the outputs of vrGazeCore to a scientific question

By the end of this tutorial, you should have an understanding of how to use vrGazeCore to analyze eye-tracking data from VR environments and how its outputs can be used to answer scientific questions.

**Study paradigm** For this tutorial, we are using data from a study in which participants (N=60) were asked to look around 360-degree scenes (N=100, 16 s/scene) just as they would in real life ([Haskins et al., 2023](https://2023.ccneuro.org/view_paper.php?PaperNum=1533)). The scenes used in this study were photospheres of real-world environments, and are rich in scene content, each displaying people and objects in complex indoor and outdoor environments. To calibrate the in-headset eye-tracker, participants performed a full field calibration of the 360-degree environment. Before each Scene Trial, a Calibration Check occurred, during which participants fixated on a red circle on the horizon to check the calibration of the in-headset eye-tracker.

Today we'll be looking at a subset of the data from this study: 7 participants, who each looked at 6 scenes for 16 seconds.

## Section 1: Loading packages and setting parameters
*Learning Objectives: how to set paths and parameters for vrGazeCore*

First, we need to clone the GitHub repository for this tutorial, which includes the raw data from the eye-tracker, the stimuli used in the experiment, the vrGazeCore scripts, and the scene content models which we'll use later in the tutorial. We'll also load the dependencies needed to run the vrGazeCore package.

In [ ]:
# Clone the tutorial repository and import the dependent packages

!git clone https://github.com/Robertson-Lab/CCN23-Tutorial.git  # clone the repository

# import dependencies
import os, sys

import numpy as np
import glob
from matplotlib import pyplot as plt
from scipy.io import loadmat
from scipy import stats
import cv2
import pandas as pd
from IPython.display import Image
import time

# import vrGazeCore package
sys.path.insert(0, '/Users/deepasriprasad/Desktop/test/CCN23-Tutorial/')  # rename with full path to CCN23-Tutorial folder
from vrgaze.parser import get_args_parser, set_paths
from vrgaze.vrgaze import vrGazeCore, vrGazeData

import semmaps

Next, let's set some parameters. vrGazeCore is built to be customizable, so that researchers can tailor the toolbox to their own experimental paradigm (e.g. headset type, trial length), processing needs (e.g. filtering steps), and analytical needs (e.g. producing certain kinds of plots). Because of this, vrGazeCore has a lot of parameters that can be adjusted. Here, we'll highlight parameters that are critical to adjust for our experiment, but you can take a look at all of the available parameters by opening 'CCN23-Tutorial/vrgaze/parser.py'.

We will also double check that the paths have been set correctly. You'll be prompted to enter either '1' if the paths have been set correctly or '0' if they haven't.

In [ ]:
# Load parameters for today's experiment

base_dir = 'yourDirectory' # full path to the project folder

args = [
    # sets the type of headset used in the experiment(0=DK2 headsets; 1=HTC Vive; 2=HTC Vive Eye; 3=Oculus Go)
    f'--headset_type=',  # used DK2
    # sets how long each scene was viewed in seconds
    f'--scene_length=',  # Scene Trial length was 16s
    # determines how we handle binocular data (0=left eye only; 1=right eye only; 2=the best eye based on eye-tracker confidence; 3=average of both eyes)
    f'--use_eye=',  # using an average of both eyes
    f'--project_dir={base_dir}',  # sets the project directory which includes the raw data and stimuli
    f'--raw_data_folder=CCN23-Tutorial/rawData',  # sets raw data folder
    f'--stim_folder=CCN23-Tutorial/stimuli',  # sets stimuli folder
    f'--plot_fixations',  # flag to produce plots of fixations overlayed on stimuli
    f'--plot_density_maps',  # flag to produce fixation density maps
]
parser = get_args_parser() #parse the args
params = parser.parse_args(args) #save

paths = set_paths(params) #save paths

## Section 2: Where does a single subject look at in one scene?
*Learning Objective: processing a single subject and single scene*

Let's first see how a single subject looks at a scene and what parts of the scene they pay attention to. We'll be looking at how subject 'oat026' looks at the following scene (scene trial '3P_4757361674_d3090b0d6b_o') and the calibration check that comes before it!

![scene trial](https://drive.google.com/uc?export=view&id=15xVaMngXhmfMf-Lb6qlpyxiP8Xzd6S93)





### 2.1: Loading & Looking at Raw Data
We'll first load in and parse the raw data for subject 'oat026'.

In [ ]:
# Loading raw data of one subject

subject_fn = '' # name of subject's raw data file: 'oat026.txt'
subject = os.path.splitext(subject_fn)[0]  # remove '.txt' ending to get subject name
vrGaze = vrGazeCore(params, paths)  # load parameters and paths for subject
raw_data = vrGaze.loadRawData(subject_fn)  # load raw data for subject

raw_data = vrGaze.processRawData(raw_data)  # process raw data to average coordinates from both eyes

Next, let's take a look at what the raw data looks like!

In [ ]:
# Print processed raw data


As you can see, the raw data has 11948 rows and 8 columns. Each row of the raw data is a timepoint where the in-headset eye-tracker recorded data; the eye-tracker we used collected around 50 data points per second. Each column contains data:

1.   Trial name (same as the stimuli name)
2.   Timestamp since experiment start
3.   Head position (yaw)
4.   Head position (pitch)
5.   Head position (roll)
6.   Eye position (x-coordinate on screen)
7.   Eye position (y-coordinate on screen)
8.   Eye-tracker confidence level


To organize this data into discrete trials, vrGazeCore parses the raw data by the trial name and stores this data as a list of trials.

In [ ]:
# Organize raw data into trials
parsed_data = vrGaze.parseTrials(data=raw_data, subject=subject)  # parse raw data into trials
print(type(parsed_data))

If we want to know which trials were run and how to select them within the `parsed_data` list, we can use the `vrGaze.parsedDataKey` function to create a summary of which trials were run and what their index is in the list.

In [ ]:
# Create and print a summary of which trials were run in this experiment
parsed_data_key = vrGaze.parsedDataKey(parsed_data)  # create a key for the trials in the parsed data
parsed_data_key

The scenes we're using come from a large database of images, so our trial names aren't have intuitive like "scene1" but are instead strings of letters and numbers. The '_sanityTarget360_0000' that appears before each scene trial is what we call our calibration check trials.

###2.2: Examining a Calibration Check

Before any Scene Trial in our experiment, we inserted a Calibration Check. For these checks, a participant was asked to look at a red circle on the horizon line and were required to successfully look at it for 5 seconds before the Scene Trial would start. This ensured two things: 1) that eye-tracker calibration was holding strong (if not, we would re-calibrate the eyetracker before continuing the experiment), 2) that all participants started each scene trial from a common fixation point.

Let's look at gaze behavior on this Calibration Check. We can calculate the fixations during the trial and plot them over the scene.

![calibration check](https://raw.githubusercontent.com/Robertson-Lab/CCN23-Tutorial/main/stimuli/_sanityTarget360_0000.png)

We calculate fixations as time windows where the mean absolute deviation (MAD) of gaze velocity lies below a set threshold (Voloh et al. (2019), *J. eye mov. res.*). We set our threshold to 50 degrees per second to define a fixation, but you can adjust the parameter in `vrGaze.params.min_mad`.

In [ ]:
# Find Calibration Check Fixations
trial = parsed_data[2]  # load the trial's parsed data
print(f'\nRunning Subject {trial.subject}, Trial {trial.trial_name}')  # print subject and trial name
trial = vrGaze.runFindFixations(trial)  # find the fixations made on this trial

Here, you are looking at your first vrGazeCore readout! You can see that vrGazeCore has relayed some information about the fixations during this check. Unsurprisingly, there was only 1 fixation during this trial, it was ~5s long.

We can also see how different filters have been applied and some basic statistics about the fixations that occured. We'll look more into that aspect of the readout later when we analyze a scene trial.

Next, let's plot the fixation!

In [ ]:
# Plot Fixation from Calibration Check
image_path = '/yourDirectory/CCN23-Tutorial/stimuli/_sanityTarget360_0000.png'  # path to calibration check image file
vrGaze.plotFixations(trial.get_fixations(), image_path, fig_size=(12,6))  # plot fixations overlayed on image file
plt.colorbar(label = 'Duration of Fixation (seconds)') # adds colorbar to plot

The dot corresponds to the participant's fixation. As you can see, the participant fixated directly over the red circle, as instructed!

### 2.3: Examining a Scene Trial
Now, let's take a look at the actual scene that the subject viewed.

![scene trial](https://raw.githubusercontent.com/Robertson-Lab/CCN23-Tutorial/main/stimuli/3P_4757361674_d3090b0d6b_o.jpg)

Like before, let's first calculate the fixations during the scene.

In [ ]:
# Find fixations on Scene Trial
trial = parsed_data[3]  # load in scene trial's parsed raw data
print(f'\nRunning Subject {trial.subject}, Trial {trial.trial_name}')  # print subject and trial name
trial = vrGaze.runFindFixations(trial)  # find fixations in scene trial

Once again, vrGazeCore will read out some information about its analysis steps. Let's walkthrough what this readout means:
1. Confidence Filter: filters out data points below a eye-tracker confidence threshold (set in `vrGaze.params.min_conf_thresh`). Here, 16.05% of our raw data was filtered out based on this threshold.
2. Eccentricity Filter: filters out data points beyond a given degree of eccentricity (in the periphery) (set in `vrGaze.params.ecc_filt_x` & `vrGaze.params.ecc_filt_y)`. We did not use an eccentricity filter in this experiment.
3. Fixation Concatenation: concatenates fixations that are close in time and position (distance set in `vrGaze.params.fix_spatial_dist`; time set in `vrGaze.params.fix_temp_dist`)
4. Fixation Trim Filter: filters out the first fixation that occured in the trial, as it is biased towards the initial viewing point (set in `vrGaze.params.exclude_first_n_fix`)
5. Fixation Duration Filter: removes fixations shorter than a set threshold duration (set in `vrGaze.params.exclude_fix_durs_less_than`); our default is removing fixations shorter than 0.1 seconds


vrGazeCore also reads out some statistics on the fixations that occured during the scene! We can see that this participant made a total of 35 fixations, with an average duration of 0.215 seconds per fixation. We can plot a histogram of this data as well!



In [ ]:
# Plot histogram of fixation durations
fix_data = trial.get_fixations()  # load trial fixation data

plt.figure()  # plot histogram of duration data
plt.hist(fix_data['duration'])
plt.title(f'Distribution of Fixation Durations for {trial.trial_name}')
plt.xlabel('Fixation Duration (seconds)')
plt.ylabel('Number of Fixations')

The histogram shows that most of the fixations for this scene where between 0.1 and 0.2 seconds.

We can also look at the data for each fixation calculated!

In [ ]:
trial.get_fixations()  # gets fixation data for trial

Each row in this data frame is a fixation. For each fixation (N=35), we have the following data:
1. Fixation yaw (horizontal axis position)
2. Fixation pitch (vertical axis position)
3. Fixation start time (referenced to start of task)
4. Fixation end time (referenced to start of task)
5. Spread of data points that comprise fixation
6. Duration of fixation (in seconds)
7. Normed fixation start time (referenced to start of trial)
8. Normed fixation end time (referenced to start of trial)


vrGazeCore also saves this data as a .pkl file. You can find this output in '/content/eyeTrackResults/fixations/pkl/oat026/oat026_3P_4757361674_d3090b0d6b_o_00004.pkl'.

It's difficult to easily interpret this data as a table, but we can overlay these fixations onto the scene image to understand where these fixations actually lie.

In [ ]:
image_path = trial.get_image_path()  # get the path to the scene trial image
vrGaze.plotFixations(trial.get_fixations(), image_path, fig_size=(12,6))  # plot the fixations over the image
plt.colorbar(label = 'Duration of Fixation (seconds)')

We can see that the participant has many fixations on the people in this scene, and some fixations on the objects around the scene. The color of the dots represent the duration of the fixation, while the cross-hairs indicate the spread of the data points underlying that fixation.

We can also calculate the density of these fixations, as weighted by duration of the fixation, and plot it onto the scene.

In [ ]:
trial = vrGaze.runHeatmapping(trial)  # calculate fixation density, as weighted by duration
density_maps = trial.get_density_map()  # get the density maps for the trial
density_map = density_maps[0]  # get the first density map for the trial
normed_map = density_map/density_map.max()   # normalize density maps by maximum value to get scale from 0 to 1
vrGaze.plotFixationDensity(normed_map,image_path,start_dur = 0, end_dur = vrGaze.params.scene_length, fig_size = (12,6))  # plot the density map overlayed on the image
plt.colorbar(label = 'Fixation Density Value')

The more yellow colors indicate a higher fixation density, whereas the dark blue color indicates no fixations. You should see that the fixation density is concentrated on the young woman, and a few objects around the scene, which confirms what we saw in the fixation plot: for this scene, the participant is looking mainly at **who** is in the scene.

## Section 3: How does a participant's gaze behavior change over time?
*Learning Objective: time-segmenting fixation density maps*

So far, we've visualized this participant's gaze behavior collapsed across the entire scene trial. But it might also be useful to see how a participant's gaze position change over the viewing duration of the scene!

vrGazeCore can segment the scene duration into a number of timesteps, which divides the scene duration into equal segments of time and calculates the fixation density for each segment.

Let's try segmenting into 4 timesteps (4s each)!

In [ ]:
# Time segment the fixation density map
vrGaze.params.heatmap_timesteps =   # set the number of time segments to 4
vrGaze.params.make_density_map_gif =   # set to 1 to turn on the flag for gif making

trial = vrGaze.runHeatmapping(trial)  # calculate fixation density for the trial, with the new timestep

density_maps = trial.get_density_map()  # get the density maps calculated
timestep_bounds = np.linspace(0, vrGaze.params.scene_length, vrGaze.params.heatmap_timesteps + 1)  # get the timestep bounds for each map (in seconds)


# for each density map, plot the map overlayed on the scene image
for i in range(len(density_maps)):
  density_map = density_maps[i]  # get the density map
  start_dur = timestep_bounds[i]  # get start time of timestep
  end_dur = timestep_bounds[i+1]  # get end time of timestep
  normed_map = density_map/density_maps.max()  # normalize density maps by maximum value to get scale from 0 to 1

  vrGaze.plotFixationDensity(normed_map,image_path,start_dur = start_dur, end_dur = end_dur, vmin = 0, vmax = 1, fig_size = (12,6))  # plot fixation density map
  plt.colorbar(label = 'Fixation Density Value')  # plot colorbar

From the time segmenting, we see that this participant looked at faces for the first 4 seconds of viewing, then looked at the objects and walkways in the scene for the next 4 seconds, before returning to look at the people in the scene for the remaining duration.

vrGazeCore can also create a GIF of these fixation density maps for easy viewing!

In [ ]:
# Display GIF
fname = '/yourDirectory/eyeTrackResults/densityMaps/plots/oat026/4-timeSegments/3P_4757361674_d3090b0d6b_o/oat026_3P_4757361674_d3090b0d6b_o_00004_4-timeSegments.gif'
Image(open(fname, 'rb').read())

## Section 4: How does a group of subjects look at multiple scenes?
*Learning Objective: process a batch of subjects & run a group fixation density analysis*

In the previous section, we looked at one participant's data for one scene. But when we're answering a scientific question, we want to be able to examine the gaze data from many different scenes and participants.



### 4.1: Batch processing participants
To do this, we can find fixations for a batch of participants' raw data by looping through each file in the raw data directory.

In [ ]:
# Set the number of fixation density timesteps to 1
vrGaze.params.heatmap_timesteps =

In [ ]:
# Process a batch of participants: load the raw data, find fixations, and calculate fixation density

subject_files = os.listdir(vrGaze.paths['project_raw_data_dir'])  # get the subject files in the raw data directory
for subject in subject_files:  # for each subject in the raw data directory
  # load and parse the subjects data
  subject_fn = subject  # get subject raw data file
  subject = os.path.splitext(subject_fn)[0]  # get subject name
  vrGaze = vrGazeCore(params, paths)  # set parameters & paths
  raw_data = vrGaze.loadRawData(subject_fn)  # load raw data

  raw_data = vrGaze.processRawData(raw_data)  # process raw data
  parsed_data = vrGaze.parseTrials(data=raw_data, subject=subject)  # parse raw data into trials

  print(f'Running {len(parsed_data)} trials for Subject {trial.subject}')
  for i, trial in enumerate(parsed_data):  # for each trial in the parsed data
      print(f'\nRunning Subject {trial.subject}, Trial {trial.trial_name} (Trial Index {i})')

      trial = vrGaze.runFindFixations(trial)  # find fixations in the trial
      trial = vrGaze.runHeatmapping(trial)  #  calculate fixation density of the trial


vrGazeCore will print a long readout, like the one we saw in the previous section, for each scene and subject. You can find the fixation data and fixation density data outputted by vrGazeCore in 'contents/eyeTrackResults' for each scene and subject. Any plots generated by vrGazeCore are also saved in that directory. The files are organized into folders based on analysis type ('fixations' or 'densityMaps'), file type ('pkl' or 'plots'), and subject. Each output file is labeled with the subject name and trial name.

### 4.2: Group Fixation Density

Now that we've calculated fixations for each subject's scene data, we can create group fixation density maps for each scene!

First, let's load the data for all of our subjects!

In [ ]:
# Load all subject data
vrGaze.params.cohort_name = "tutorial-cohort"  # cohort name used to label files & directories
selected_files = os.listdir(vrGaze.paths['project_raw_data_dir'])  # subject directory to use in group data analysis
subjects = [os.path.splitext(subject)[0] for subject in selected_files]  # get subject names by removing '.txt' from file names
group_trials = vrGaze.loadGroupFixations(subjects)  # load scenes that are shared by all participants

vrGazeCore once again saves the group data as a list. We can create a key for that list to see the indices for each subject and scene using `vrGaze.parsedDataKey()`.

In [ ]:
# Create a key for the group data
group_key = vrGaze.parsedDataKey(group_trials)
group_key  # view group data key

Since we don't want to calculate the fixation density for our Calibration Check trials, let's first remove them from our group data.

In [ ]:
# Write some code! Remove calibration checks from fixation density analysis by finding the index of '_sanityTarget360_0000' trials
del group_trials[]  # insert scene_index of '_sanityTarget360_0000' trials
group_key = vrGaze.parsedDataKey(group_trials)  # check that '_sanityTarget360_0000' trials has been removed from group_trials
group_key

Next, we can run fixation density calculations for each scene!

In [ ]:
# Calculate fixation density maps for group

for trials in group_trials:  # for each scene trial

    group_data = vrGaze.runHeatmapping(trials)  # run fixation density mapping using all subject's fixations

    density_maps = group_data.get_density_map()  # get the fixation density map
    density_map = density_maps[0]

    start_dur = timestep_bounds[0]  # get the start time of the density map
    end_dur = vrGaze.params.scene_length  # get the end time of the density map
    image_path = group_data.image_path  # get the trial image
    normed_map = density_map/density_maps.max()  # normalize density maps by maximum value to get scale from 0 to 1

    vrGaze.plotFixationDensity(normed_map,image_path,start_dur = start_dur, end_dur = end_dur, fig_size = (12,6))  # plot the density map overlayed on the trial image
    plt.colorbar(label = 'Fixation Density Values')

Although all these scenes differ in their content, we can see that there's a high group fixation density in the areas of the scene that contain faces.

## Section 5: Using vrGazeCore outputs: Comparing group gaze data to models of scene content
*Learning Objective: Using outputs of vrGazeCore to answer a scientific question*

In this section, we're going to see an example of how vrGazeCore outputs can be used to examine a scientific question.

Let's say we're interested the following question:
> How do we allocate our attention when we step into a new, real-world environment? In particular, in what order do we prioritize scene-content that will inform us about **where** we are vs. **who (people)** is here vs. **what (objects)** is here?

VR is an excellent way to examine this question, as it allows participants to move their heads and eyes around a new environment and set their own priorities much like they would in real life!

We can get a a qualitative sense of how subjects are looking at scenes over time by just looking at where their fixations lie on the scenes, but we can compare these fixations to models to get a *quantitative answer* to the question of how people allocate their attention when viewing a scene.

The models we're using in this tutorial are "semantic maps" of the who, what, and where information in each scene. To generate these maps, we decomposed each scene into smaller image tiles and crowdsourced verbal descriptions for each tile. We then used the computational language model BERT to transform those descriptions into three semantic values, representing the amount of 'who', 'what', and 'where' information respectively (Haskins et al., *2022*, *Autism Research*).

### 5.1: Examining Content Maps

Let's first see what content maps look like. We'll be looking at the semantic information for the following image:

![scene trial](https://drive.google.com/uc?export=view&id=1E9oykbuHtpohTwO2LgRrObMqFpI1XpHy)

For the scene, we have three content maps:

1. 'What' map: distribution of the object information in the scene
2. 'Where' map: distribution of the navigational affordance (e.g., doors, paths,etc.) information in the scene
3. 'Who' map: distribution of the faces/people information in the scene

Each content map has a value for every pixel of the scene, indicating how much of that content that pixel represents.

Let's plot these maps over the scene image to get a better sense of how they look!

In [ ]:
# load the scene content map directory
map_dir = os.path.join(params.project_dir,'CCN23-Tutorial/sceneContentMaps')  # set the scene content map directory
scene_name = '36154312510_39f372cb1e_o'  # set scene name
image_path = os.path.join(params.project_dir, params.stim_folder,scene_name + '.jpg')  # get path to scene image

# load the scene content maps
who_array = semmaps.read_sem_map(os.path.join(map_dir,scene_name + '_who.mat'))  # load who scene content map of scene
what_array = semmaps.read_sem_map(os.path.join(map_dir,scene_name + '_what.mat'))  # load what scene content map of scene
where_array = semmaps.read_sem_map(os.path.join(map_dir,scene_name + '_where.mat'))  # load where scene content map of scene

# plot the scene content maps
semmaps.plot_sem_map(who_array,image_path,vrGaze.params.plotting_image_width,vrGaze.params.plotting_image_height, map_type = 'Who', fig_size = (12,6))
semmaps.plot_sem_map(what_array,image_path,vrGaze.params.plotting_image_width,vrGaze.params.plotting_image_height, map_type = 'What', fig_size = (12,6))
semmaps.plot_sem_map(where_array,image_path,vrGaze.params.plotting_image_width,vrGaze.params.plotting_image_height, map_type = 'Where', fig_size = (12,6))

For these maps, blue indicates a low semantic value while yellow indicates a high semantic value. We can see that each map shows a high semantic value over the kind of information they represent. For example, the 'Who' scene content map has high semantic values where the faces are in the image.

### 5.2 Comparing Fixations to Content Maps

To get a quantitative answer to our question, we can use the positions of the fixations determined by vrGazeCore and look at semantic value at those positions for each of these three maps. We can then plot this over the course of the initial fixations to see how to information the fixations represent changes over time.

For this tutorial, let's look at the first 15 fixations. We'll get the values of each map at these fixation positions and plot how the semantic values change over the course of the fixations.

In [ ]:
# Running content map comparisons

selected_files = ['furrow397_V1.txt', 'oat026.txt', 'oat031.txt', 'furrow441_v1.txt', 'oat073.txt', 'furrow394_V1.txt', 'furrow499_V1.txt']  # get list of subjects to use
subjects = [os.path.splitext(subject)[0] for subject in selected_files]  # get subject names by removing file endings
group_trials = vrGaze.loadGroupFixations(subjects)  # load subject fixations for all scenes


group_map_data = pd.DataFrame()  # create empty data frame to store semantic values
for trials in group_trials:  # for each scene trial of group
  for trial in trials:  # for each subject
    # ignore if calibration check
    if any([item in trial.trial_name for item in vrGaze.params.pretrial_list]):
            print (f'Skipping {trial.trial_name} - not a scene!')
            continue
    map_data = semmaps.run_sem_map_comparison(trial, map_dir,vrGaze.params.plotting_image_width,vrGaze.params.plotting_image_height,15)  # compare first 15 fixations to each scene content map and get semantic values
    group_map_data = pd.concat([group_map_data, map_data])  # append values to group semantic values

group_analysis = group_map_data.groupby(['map_type', group_map_data.index]).agg({'sem_vals':[np.nanmean],'weighted_sem_vals':[np.nanmean],'over_max':[np.nanmean],'weighted_over_max':[np.nanmean]})    # calculate mean semantic value for each fixation for each map

# separate mean semantic values by map type
who_data = group_analysis['weighted_over_max'].loc['who']
what_data = group_analysis['weighted_over_max'].loc['what']
where_data = group_analysis['weighted_over_max'].loc['where']

# plotting mean semantic values by fixation for each map type
plt.figure()
data1 = plt.errorbar(who_data.index + 1, who_data['nanmean'], stats.sem(who_data), fmt='o', linewidth=2, capsize=6)
data2= plt.errorbar(what_data.index + 1, what_data['nanmean'], stats.sem(what_data), fmt='o', linewidth=2, capsize=6)
data3 = plt.errorbar(where_data.index + 1, where_data['nanmean'], stats.sem(where_data), fmt='o', linewidth=2, capsize=6)
plt.title('Semantic Values for First 15 Fixations')
plt.xlabel('Fixation Number')
plt.ylabel('Semantic Map Value')
plt.legend([data1, data2, data3], ['Who', 'What','Where'])

Here we see that subjects primarily direct their attention to 'Who' information when they first enter a scene!

## Conclusion & Take Aways

In this tutorial, you learned how to use vrGazeCore, an open source toolbox for eye-tracking analysis in VR.

Along the way, you discovered that, when people step into a novel, real-world environment, they first allocate their attention to scene content that informs them about **who** is in the scene (i.e., people), *before* examining **what** is in the scene (i.e., objects) or **where** they are (i.e. paths).

You're now equipped to analyze VR eye-tracking data of your own! To do that, please visit our [GitHub repository](https://github.com/Robertson-Lab/vrGazeCore-Toolbox) and cite our [CCN paper](https://2023.ccneuro.org/view_paper.php?PaperNum=1555).